In [68]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import pickle

import random

In [40]:
df = pd.read_csv('tran_prod_cust .csv')

C:\Windows\Temp\ipykernel_29948\1606241303.py:1: DtypeWarning: Columns (17,18,19,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('tran_prod_cust .csv')


In [41]:
df.drop(['GivenName','Surname','StreetAddress','ZipCode', 'City','PRODUCT_NAME', 'CATEGORY', 'COLOR_CODE', 'TITLE','PRODUCT_TYPE'], axis = 1, inplace = True)

In [42]:
df.head()

,DEPARTMENT,SUBCATEGORY,IS_BESTSELLER,COLOR,PRICE_CURRENT,UserId,TransactionId,TransactionTime,ItemCode,NumberOfItemsPurchased,PRICE_RETAIL,revenue,State,Birthday,Age
0,Kids,Hats,False,Light Menta/Barely Green,16.95,185792,60595023,2/17/2019 8:18,12231348,10,16.95,169.50,WA,9/23/1986,37.0
1,Kids,Hats,False,Light Menta/Barely Green,16.95,108854,12058664,7/24/2018 10:46,12231348,6,16.95,101.70,FL,8/6/1945,78.0
2,Kids,Hats,False,Light Menta/Barely Green,16.95,193153,38728736,2/21/2018 7:42,12231348,9,16.95,152.55,PA,4/24/1975,48.0
3,Kids,Hats,False,Light Menta/Barely Green,16.95,151472,50903069,10/12/2018 10:39,12231348,10,16.95,169.50,NY,9/16/1998,25.0
4,Kids,Hats,False,Light Menta/Barely Green,16.95,196977,95424449,1/29/2019 9:16,12231348,10,16.95,169.50,CA,3/24/1980,44.0


In [43]:
# Initialize LabelEncoders
encoder_files = [
    'department_encoder.pkl',
    'subcategory_encoder.pkl',
    'color_encoder.pkl',
    'state_encoder.pkl',
    'is_bestseller_encoder.pkl'
]

loaded_encoders = {}

for file_name in encoder_files:
    with open(file_name, 'rb') as f:
        encoder = pickle.load(f)
        loaded_encoders[file_name.split('.')[0]] = encoder

# Transform DataFrame using loaded encoders
df['DEPARTMENT'] = loaded_encoders['department_encoder'].transform(df['DEPARTMENT'])
df['SUBCATEGORY'] = loaded_encoders['subcategory_encoder'].transform(df['SUBCATEGORY'])
df['COLOR'] = loaded_encoders['color_encoder'].transform(df['COLOR'])
df['State'] = loaded_encoders['state_encoder'].transform(df['State'])
df['IS_BESTSELLER'] = loaded_encoders['is_bestseller_encoder'].transform(df['IS_BESTSELLER'])


In [44]:
df['TransactionTime'] = pd.to_datetime(df['TransactionTime'])
df['month'] = df['TransactionTime'].dt.month
df['day'] = df['TransactionTime'].dt.day

In [45]:
df.drop(['TransactionTime', 'Birthday', 'TransactionId', 'UserId', 'PRICE_CURRENT','NumberOfItemsPurchased','PRICE_CURRENT', 'ItemCode'], axis = 1, inplace = True)

In [46]:
df.head()

,DEPARTMENT,SUBCATEGORY,IS_BESTSELLER,COLOR,PRICE_RETAIL,revenue,State,Age,month,day
0,0,6,0,1529,16.95,169.50,48,37.0,2,17
1,0,6,0,1529,16.95,101.70,9,78.0,7,24
2,0,6,0,1529,16.95,152.55,39,48.0,2,21
3,0,6,0,1529,16.95,169.50,35,25.0,10,12
4,0,6,0,1529,16.95,169.50,4,44.0,1,29


In [48]:
df.dropna(inplace = True)

In [50]:
X = df.drop('revenue', axis = 1)
y = df['revenue']

In [51]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [52]:
X_train.shape, y_train.shape

((648432, 9), (648432,))

In [53]:
model = LinearRegression()

model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)


In [54]:
print(f"Mean Squared Error: {mse:.2f}")
print(f"R^2 Score: {r2:.2f}")

Mean Squared Error: 101271.46
R^2 Score: 0.57


# Without Product Price

In [55]:
ddf = df.copy()

In [56]:
ddf.drop('PRICE_RETAIL', axis = 1, inplace = True)

In [57]:
X1 = ddf.drop('revenue', axis = 1)
y1 = ddf['revenue']

X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.2, random_state=42)

model_2 = LinearRegression()

model_2.fit(X1_train, y1_train)

y1_pred = model_2.predict(X1_test)

mse_1 = mean_squared_error(y1_test, y1_pred)
r2_1 = r2_score(y1_test, y1_pred)

print(f"Mean Squared Error: {mse_1:.2f}")
print(f"R^2 Score: {r2_1:.2f}")

Mean Squared Error: 228722.63
R^2 Score: 0.03


In [58]:
with open('has_price.pkl', 'wb') as f:
    pickle.dump(model, f)

with open('no_price.pkl', 'wb') as f:
    pickle.dump(model_2, f)

In [74]:
new_df = pd.read_csv('tran_prod_cust .csv')
new_df.dropna(inplace = True)

C:\Windows\Temp\ipykernel_29948\2542132129.py:1: DtypeWarning: Columns (17,18,19,20,21,23) have mixed types. Specify dtype option on import or set low_memory=False.
  new_df = pd.read_csv('tran_prod_cust .csv')


In [75]:
random.seed(42)

test_data = new_df.sample(1)
test_data

,DEPARTMENT,CATEGORY,SUBCATEGORY,PRODUCT_NAME,TITLE,PRODUCT_TYPE,IS_BESTSELLER,COLOR,PRICE_CURRENT,COLOR_CODE,...,PRICE_RETAIL,revenue,GivenName,Surname,StreetAddress,City,State,ZipCode,Birthday,Age
18069,Kids,Accessories and Equipment,All Accessories and Equipment,Nike Pro,Kids' Hijab,EQUIPMENT,False,Black/White,25.95,N0003709-010,...,25.95,181.65,Clarence,Barnes,3893 Conference Center Way,Mifflinburg,PA,17844.0,4/4/1957,66.0


In [76]:
test_data['TransactionTime'] = pd.to_datetime(test_data['TransactionTime'])
test_data['month'] = test_data['TransactionTime'].dt.month
test_data['day'] = test_data['TransactionTime'].dt.day

columns_to_keep = ['DEPARTMENT', 'SUBCATEGORY', 'IS_BESTSELLER', 'COLOR', 'PRICE_RETAIL', 'State', 'Age', 'month', 'day']
test_data = test_data[columns_to_keep]
test_data

,DEPARTMENT,SUBCATEGORY,IS_BESTSELLER,COLOR,PRICE_RETAIL,State,Age,month,day
18069,Kids,All Accessories and Equipment,False,Black/White,25.95,PA,66.0,7,23


In [77]:
test_data['DEPARTMENT'] = loaded_encoders['department_encoder'].transform(test_data['DEPARTMENT'])
test_data['SUBCATEGORY'] = loaded_encoders['subcategory_encoder'].transform(test_data['SUBCATEGORY'])
test_data['COLOR'] = loaded_encoders['color_encoder'].transform(test_data['COLOR'])
test_data['State'] = loaded_encoders['state_encoder'].transform(test_data['State'])
test_data['IS_BESTSELLER'] = loaded_encoders['is_bestseller_encoder'].transform(test_data['IS_BESTSELLER'])

C:\Windows\Temp\ipykernel_29948\1722564147.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['DEPARTMENT'] = loaded_encoders['department_encoder'].transform(test_data['DEPARTMENT'])
C:\Windows\Temp\ipykernel_29948\1722564147.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['SUBCATEGORY'] = loaded_encoders['subcategory_encoder'].transform(test_data['SUBCATEGORY'])
C:\Windows\Temp\ipykernel_29948\1722564147.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slic

In [78]:
test_data

,DEPARTMENT,SUBCATEGORY,IS_BESTSELLER,COLOR,PRICE_RETAIL,State,Age,month,day
18069,0,0,0,591,25.95,39,66.0,7,23


In [79]:
# Original revenue: 181.65
model.predict(test_data)

array([165.28403478])